<a href="https://colab.research.google.com/github/hirokimituya/stock-price-analysis/blob/main/technical_analysis/%E6%A0%AA%E4%BE%A1%E5%88%86%E6%9E%90%E9%96%A2%E6%95%B0_backtesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install backtesting

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173817 sha256=8461d3428840ad37861ba3d0857634fe63813c819ccc8d78a6f76a63f210d685
  Stored in directory: /root/.cache/pip/wheels/e2/30/7f/19cbe31987c6ebdb47f1f510343249066711609e3da2d57176
Successfully built backtesting


In [3]:
import pandas_datareader.data as pdr

# 株価データを取得するメソッド
def get_stock_data(code):
    """株価データを取得する

    :param code: 取得する株価データの銘柄コード
    :return: 株価データのデータフレーム
    """
    df = pdr.DataReader(f'{code}.JP', 'stooq').sort_index()
    return df

In [6]:
# バックテストの実行と結果の確認
from backtesting import Backtest
import datetime as dt

def run_backtest(strategy, code, start, end=dt.date.today()):
    """バックテストを実行する

    :param strategy: バックテストで用いる売買ルール
    :param code: 取得する株価データの銘柄コード
    :param start: 取得する株価データの開始日
    :param end: 取得する株価データの終了日
    """
    df = get_stock_data(code)
    data = df[start:end]

    bt = Backtest(data, strategy, trade_on_close=True)

    # バックテスト実行
    # result = bt.run()

    result = bt.optimize(ns=range(5, 25, 5), nl=range(5, 75, 5), maximize='Return [%]', constraint=lambda r: r.ns < r.nl)

    # 実行結果のデータを表示
    print(result)

    # 実行結果をグラフで表示
    bt.plot()

In [7]:
# 単純移動平均のゴールデンクロス／デッドクロスを用いた売買ルール
from backtesting import Strategy
from backtesting.lib import crossover
from backtesting.test import SMA

class SmaCross(Strategy):
    ns = 5  # 短期移動平均日数
    nl = 25 # 長期移動平均日数

    def init(self):
        # 短期移動平均
        self.smaS = self.I(SMA, self.data['Close'], self.ns)
        # 長期移動平均
        self.smaL = self.I(SMA, self.data['Close'], self.nl)
    
    def next(self):
        # smaS > smaL で買う
        if crossover(self.smaS, self.smaL):
            self.buy()
        # smaS < smaL で売る
        elif crossover(self.smaL, self.smaS):
            self.position.close()

In [8]:
run_backtest(SmaCross, 2395, '2021-1-1', '2022-3-31')

Backtest.optimize:   0%|          | 0/2 [00:00<?, ?it/s]

Start                     2021-01-04 00:00:00
End                       2022-03-31 00:00:00
Duration                    451 days 00:00:00
Exposure Time [%]                   57.894737
Equity Final [$]                    22827.953
Equity Peak [$]                     23476.523
Return [%]                          128.27953
Buy & Hold Return [%]              156.444719
Return (Ann.) [%]                   98.221269
Volatility (Ann.) [%]               79.721742
Sharpe Ratio                         1.232051
Sortino Ratio                        4.185057
Calmar Ratio                         3.968653
Max. Drawdown [%]                  -24.749272
Avg. Drawdown [%]                   -6.105125
Max. Drawdown Duration      132 days 00:00:00
Avg. Drawdown Duration       24 days 00:00:00
# Trades                                    3
Win Rate [%]                            100.0
Best Trade [%]                      74.132251
Worst Trade [%]                     11.321005
Avg. Trade [%]                    